# Google Cloud 

This notebook retrieves the top 500 models for Google Cloud and Vertex AI Garden including TGI configurations. 

In [4]:
from tqdm import tqdm
from dataclasses import asdict
import requests as r 
from recommender.main import get_memory_per_model_and_tgi, get_tgi_config, get_quantization_type
from recommender.utils.const import GOOGLE_CLOUD_INFERENCE_INSTANCE_TYPES
from huggingface_hub import HfFolder

headers = {"Authorization": f"Bearer {HfFolder.get_token()}"}

sess = r.Session()
sess.headers.update(headers)


def check_for_gate(model_id):
  url = f"https://huggingface.co/api/models/{model_id}"
  response = sess.get(url).json()
  error = response.get("error",None)
  if error and "gate" in error:
    return True
  
  return False


def get_tgi_models_and_parse(limit=250,type="likes30d",filter="text-generation-inference"):
  url=f"https://huggingface.co/api/models?sort={type}&direction=-1&filter={filter}&limit={limit}"
  # url = "https://huggingface.co/api/models/meta-llama/Llama-2-7b-chat-hf"
  # response = [r.get(url, headers=headers).json()]
  response = sess.get(url).json()
  # map, filter list to remove gguf 
  filtered_models=[]
  for model in tqdm(response,total=len(response)):
    try:
      # backlist models which leads to crashes on my mac
      if model["id"] in ["LargeWorldModel/LWM-Text-Chat-1M","LargeWorldModel/LWM-Text-1M","LargeWorldModel/LWM-Text-512K", "LargeWorldModel/LWM-Chat-512K"]:
        continue
      
      # remove gguf models
      if "gguf" in model["tags"]:
        continue
      
      # check for gate
      gated = check_for_gate(model["id"])    
      
      # get license 
      license_value = next((tag.split(':', 1)[1] for tag in model["tags"] if tag.startswith('license:')), "N/A")
                  
      # create TGI recommendation for Google Cloud 
      # potential_configs = []
      for config in GOOGLE_CLOUD_INFERENCE_INSTANCE_TYPES["gpu"]:
        tgi_config = get_tgi_config(
          model_id=model["id"],
          gpu_memory=config["memoryInGB"],
          num_gpus=config["numGpus"],
        )
        if tgi_config:
          config_dict = asdict(tgi_config)
          config_dict["gcp_instance"] = config["name"]
          break # break after first valid config to get smallest instance
          # potential_configs.append(config_dict)

            
      # model size   
      filtered_models.append({
        "model_id": model["id"],
        "url": f"https://huggingface.co/{model['id']}",
        "cotaniner": "PyTorch TGI GPU",
        "license": license_value,
        "gated": gated,
        "private": model["private"],
        "configuration": config_dict,
        "likes": model["likes"],
        "likes30d": model["likes30d"],
        "downloads": model["downloads"],
        
      })
    except Exception as e:
      print(f"Error parsing model {model['id']}")
      continue
  return filtered_models



In [5]:
response = get_tgi_models_and_parse(limit=500)


  0%|          | 0/500 [00:00<?, ?it/s]

Loading pretrained config for `mistralai/Mixtral-8x7B-Instruct-v0.1` from `transformers`...
Loading pretrained config for `mistralai/Mixtral-8x7B-Instruct-v0.1` from `transformers`...
Loading pretrained config for `mistralai/Mixtral-8x7B-Instruct-v0.1` from `transformers`...
Loading pretrained config for `mistralai/Mixtral-8x7B-Instruct-v0.1` from `transformers`...


  1%|          | 4/500 [00:15<31:48,  3.85s/it]

Loading pretrained config for `bigcode/starcoder2-15b` from `transformers`...
Loading pretrained config for `bigcode/starcoder2-15b` from `transformers`...


  1%|          | 5/500 [00:23<40:20,  4.89s/it]

...

tokenizer_config.json: 100%|██████████| 727/727 [00:00<00:00, 564kB/s]
tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 13.8MB/s]
tokenizer.json: 100%|██████████| 1.84M/1.84M [00:00<00:00, 11.3MB/s]
special_tokens_map.json: 100%|██████████| 435/435 [00:00<00:00, 225kB/s]


Loading pretrained config for `cognitivecomputations/Wizard-Vicuna-7B-Uncensored` from `transformers`...


config.json: 100%|██████████| 1.07k/1.07k [00:00<00:00, 336kB/s]
tokenizer_config.json: 100%|██████████| 727/727 [00:00<00:00, 576kB/s]
tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 10.9MB/s]
tokenizer.json: 100%|██████████| 1.84M/1.84M [00:00<00:00, 10.6MB/s]
added_tokens.json: 100%|██████████| 21.0/21.0 [00:00<00:00, 14.6kB/s]
special_tokens_map.json: 100%|██████████| 435/435 [00:00<00:00, 347kB/s]


Loading pretrained config for `TheBloke/Wizard-Vicuna-13B-Uncensored-SuperHOT-8K-GPTQ` from `transformers`...


100%|██████████| 500/500 [1:04:33<00:00,  7.75s/it]


In [7]:
from datasets import Dataset 

ds = Dataset.from_list(response)

ds.to_csv("tgi_models.csv")


Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 97.23ba/s]


160148

# Embedding Models

In [8]:
def get_emb_models_and_parse(limit=25,type="likes30d",filter="sentence-transformers"):
  url=f"https://huggingface.co/api/models?sort={type}&direction=-1&filter={filter}&limit={limit}"
  response = r.get(url, headers=headers).json()
  # supported TEI architectures
  architectures = ["bert", "roberta", "xlm-roberta","nomic_bert"]
  # map, filter list to remove gguf 
  filtered_models=[]
  for model in response:
    try:
      # backlist models which leads to crashes on my mac
      if model["id"] in []:
        continue
      # filter TEI supported architectures
      if not any(architecture in model["tags"] for architecture in architectures):
        continue
      
      # get license 
      license_value = next((tag.split(':', 1)[1] for tag in model["tags"] if tag.startswith('license:')), "N/A")
      
      # check for gate
      gated = check_for_gate(model["id"])    
                  
      # model size   
      filtered_models.append({
        "model_id": model["id"],
        "url": f"https://huggingface.co/{model['id']}",
        "cotaniner": "PyTorch TEI CPU/GPU",
        "license": license_value,
        "gated": gated,
        "private": model["private"],
        "likes": model["likes"],
        "likes30d": model["likes30d"],
        "downloads": model["downloads"],
      })
    except Exception as e:      
      print(f"Error parsing model {model['id']}")
      continue
  return filtered_models  
  

In [9]:
embedding = get_emb_models_and_parse(limit=50)


In [11]:
from datasets import Dataset 

emb_ds = Dataset.from_list(embedding)

emb_ds.to_csv("emb_models.csv")


Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 480.17ba/s]


5781